**Подготовка к обучению моделей**

Импорт необходимых библиотек

In [16]:
import joblib
import pandas as pd
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

Загрузка данных и отбор признаков

In [22]:
# загрузка датасета и удаление session_id
dataframe = pd.read_csv('data/dataset.csv')
# убраны: идентификатор сессии и фичи с высокой корреляцией к таргету
dataframe.drop(columns=['session_id'], inplace=True)
dataframe.drop(columns=['quiz_show'], inplace=True)
dataframe.drop(columns=['showed_number_ads'], inplace=True)

x = dataframe.drop('custom_target', axis=1)
y = dataframe['custom_target']

# для повторяемости
random_state = 77
#  разделяем  выборку на тестовую (20%), обучающую (60%) и валидационную (20%)
x_train_val, x_test, y_train_val, y_test = train_test_split(
    x, y,
    test_size=0.2,
    stratify=y,
    random_state=random_state
)

x_train, x_val, y_train, y_val = train_test_split(
    x_train_val, y_train_val,
    test_size=0.25,  # четверть обучающей выборки
    stratify=y_train_val,
    random_state=random_state
)

**Baseline**

In [23]:
dummy = DummyClassifier(strategy="prior")
dummy.fit(x_train, y_train)
roc_auc = roc_auc_score(y_test, dummy.predict_proba(x_test)[:, 1])
print(f"Значение метрики ROC-AUC для Baseline: {roc_auc:.4f}")

Значение метрики ROC-AUC для Baseline: 0.5000


**Логистическая регрессия**

В качестве первой модели мы взяли логистическую регрессию. Будем использовать прямое кодирование (One Hot Encoding) для категориальных признаков и стандартизацию (Standard Scaler) для числовых

In [33]:
model = LogisticRegression(
    class_weight='balanced',
    random_state=random_state
)

Обучение модели

In [ ]:
model.fit(x_train_val, y_train_val)

Оценка модели линейной регрессии (ROC-AUC)

In [26]:
roc_auc = roc_auc_score(y_test, model.predict_proba(x_test)[:, 1])
print(f"Значение метрики ROC-AUC для линейной регрессии: {roc_auc}")

Значение метрики ROC-AUC для линейной регрессии: 0.8680670400440691


Сохранение модели

In [27]:
joblib.dump(model, 'model/logistic_regression.joblib')

['model/logistic_regression.joblib']

**Градиентный бустинг**

In [32]:
params = {
    'n_estimators': 150,
    'learning_rate': 0.2,
    'max_depth': 3,
    'min_samples_split': 20,
    'min_samples_leaf': 10,
    'max_features': 'sqrt',
    'subsample': 0.8,
    'random_state': random_state
}

model = GradientBoostingClassifier(**params)

Обучение модели

In [ ]:
model.fit(x_train_val, y_train_val)

Оценка модели линейной регрессии (ROC-AUC)

In [30]:
y_pred = model.predict(x_test)
roc_auc = roc_auc_score(y_test, y_pred)
print(f"Значение метрики ROC-AUC для градиентного бустинга: {roc_auc}")

Значение метрики ROC-AUC для градиентного бустинга: 0.7328495901503762


Сохранение модели

In [31]:
joblib.dump(model, 'model/gradient_boosting.joblib')

['model/gradient_boosting.joblib']